<a href="https://colab.research.google.com/github/abdullahiabubakarumar21-cloud/Aumar-tech/blob/main/Brain_Tumor_MRI_Classification_Using_Transfer_Learning_%2B_Quantum_Circuit_Layeripynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LIBRARY IMPORTATION

In [3]:
!pip install tensorflow pennylane pennylane-lightning


In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import pennylane as qml
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

IMAGE PREPROCESSING

In [6]:
img_size = 224
batch_size = 32

train_gen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True
)

test_gen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input
)

train_data = train_gen.flow_from_directory(
    "/content/drive/MyDrive/AI DATASETS/Brain tumor/Training",
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode="categorical"
)

test_data = test_gen.flow_from_directory(
    "/content/drive/MyDrive/AI DATASETS/Brain tumor/Testing",
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode="categorical"
)


Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


TRANSFER LEARNING

In [7]:
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(img_size, img_size, 3)
)

base_model.trainable = False  # Freeze pretrained layers


REDUCE FEATURES FOR QUANTUM INPUT

In [8]:
n_qubits = 4


Quantum Device

In [9]:
dev = qml.device("default.qubit", wires=n_qubits)


Parameterized Quantum Circuit (PQC)

In [10]:
@qml.qnode(dev, interface="tf")
def quantum_circuit(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]


Quantum Keras Layer

In [15]:
!pip install tensorflow pennylane pennylane-lightning --upgrade --force-reinstall

  Using cached tensorflow-2.20.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.5 kB)
  Using cached pennylane-0.43.1-py3-none-any.whl.metadata (11 kB)
  Using cached pennylane_lightning-0.43.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (11 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.12.19-py2.py3-none-any.whl.metadata (1.0 kB)
  Using cached gast-0.7.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached protobuf-6.33.2-cp39-abi3-manylinux2014_x86_64.whl.metadata (593 bytes)
  Using cached requests-2.32.5-py3-none-any.whl.metadata

In [14]:
from pennylane.qnn.keras import KerasLayer

inputs = layers.Input(shape=(img_size, img_size, 3))

x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(n_qubits, activation='relu')(x)  # feature compression

# Define the weight shapes for the quantum layer
weight_shapes = {"weights": (3, n_qubits)} # Assuming 3 layers for BasicEntanglerLayers

# Create the quantum Keras layer
quantum_layer = KerasLayer(quantum_circuit, weight_shapes, name="quantum_layer")

x = quantum_layer(x)

outputs = layers.Dense(4, activation='softmax')(x)

model = models.Model(inputs=inputs, outputs=outputs)

ModuleNotFoundError: No module named 'pennylane.qnn.keras'

In [18]:
try:
    from pennylane.qnn.keras import KerasLayer
    print("KerasLayer is successfully importable.")
except ImportError as e:
    print(f"KerasLayer is NOT importable. Error: {e}")
except AttributeError as e:
    print(f"KerasLayer is NOT importable due to AttributeError: {e}")

KerasLayer is NOT importable. Error: No module named 'pennylane.qnn.keras'


In [13]:
import pennylane as qml
print(f"PennyLane version: {qml.__version__}")

PennyLane version: 0.43.1


# Task
Check the installed Keras version and provide details on the compatibility between the installed PennyLane and Keras versions regarding the `KerasLayer` integration.

## Check Keras Version

### Subtask:
Check the installed Keras version to understand the environment's current state.


**Reasoning**:
To check the Keras version, I need to import the Keras library and then access its `__version__` attribute.



In [19]:
import keras
print(f"Keras version: {keras.__version__}")

Keras version: 3.13.0


Based on the `ModuleNotFoundError: No module named 'pennylane.qnn.keras'` and the installed Keras version `3.13.0`, it appears there is an incompatibility issue. PennyLane's `KerasLayer` from `pennylane.qnn.keras` is typically designed to work with TensorFlow's Keras (i.e., `tf.keras`) which is based on Keras 2.x.

Keras 3.x (`keras.__version__` is `3.13.0`) introduced significant architectural changes, making it a standalone library that can run on top of multiple backends (JAX, PyTorch, TensorFlow). The `pennylane.qnn.keras` module has not yet been updated to support Keras 3.x directly, leading to the `ModuleNotFoundError` when trying to import `KerasLayer`.

To resolve this, you would generally need to either:
1. Downgrade Keras to a 2.x version that is compatible with `tf.keras` (which PennyLane's `KerasLayer` expects).
2. Use a different PennyLane integration method for Keras 3.x, if one becomes available in newer PennyLane versions, or integrate quantum circuits manually into a Keras 3.x model.

For the purpose of this notebook, using Keras 2.x via `tf.keras` is the recommended path for `pennylane.qnn.keras.KerasLayer`.

## Explain Compatibility

### Subtask:
Provide details on the compatibility between the installed PennyLane and Keras versions, especially concerning the `KerasLayer` integration.


The `ModuleNotFoundError: No module named 'pennylane.qnn.keras'` strongly indicates an incompatibility issue between the installed PennyLane and Keras versions. Here's a detailed explanation:

1.  **PennyLane's `KerasLayer` and `tf.keras` (Keras 2.x)**:
    *   Historically, PennyLane's `qml.qnn.keras.KerasLayer` was designed to integrate seamlessly with `tf.keras`, which is the Keras API integrated within TensorFlow. This typically refers to Keras versions prior to 3.x, often bundled with TensorFlow 2.x releases.
    *   The `KerasLayer` utility relies on the internal structure and APIs of `tf.keras` to create quantum-classical hybrid models.

2.  **Keras 3.x Architectural Changes**:
    *   Keras 3.x (as installed, version `3.13.0`) represents a significant architectural shift. It is now a standalone deep learning framework that can run on multiple backends (TensorFlow, JAX, PyTorch).
    *   This decoupling from `tf.keras` means that the underlying structure and certain APIs that `pennylane.qnn.keras` expected are no longer present or have changed dramatically in Keras 3.x.
    *   As a result, PennyLane's `KerasLayer` (which was built for the `tf.keras` integration) is not compatible with the new Keras 3.x architecture, leading to the `ModuleNotFoundError` when trying to import it from `pennylane.qnn.keras`.

3.  **Potential Resolutions**:
    *   **Downgrade Keras to 2.x (tf.keras compatible)**: The most straightforward solution is to uninstall Keras 3.x and install a version of TensorFlow that includes a compatible `tf.keras` (Keras 2.x). For example, installing `tensorflow==2.x.x` (where `2.x.x` is a specific version) would likely resolve the `ModuleNotFoundError` by reverting to the Keras API that PennyLane's `KerasLayer` is designed to work with.
    *   **Manual Integration for Keras 3.x**: If using Keras 3.x is essential, a direct integration using `qml.keras.SoleLayer` (or similar low-level Keras utilities) might be possible. This would involve manually defining the quantum circuit as a Keras Layer without relying on the higher-level `qml.qnn.keras.KerasLayer` utility. This approach requires more manual setup but offers flexibility.
    *   **Check for PennyLane Updates**: Keep an eye on PennyLane's releases. Future versions of PennyLane might introduce a `KerasLayer` specifically designed to be compatible with Keras 3.x and its multi-backend architecture. However, as of PennyLane 0.43.1, this compatibility does not appear to be natively supported for Keras 3.x.

## Final Task

### Subtask:
Summarize the findings regarding PennyLane and Keras compatibility.


## Summary:

### Q&A
**What is the compatibility between the installed PennyLane and Keras versions regarding the `KerasLayer` integration?**
There is an incompatibility issue. The installed Keras version is 3.13.0, while PennyLane's `KerasLayer` from `pennylane.qnn.keras` is designed to work with TensorFlow's Keras (`tf.keras`), which is based on Keras 2.x. Keras 3.x introduced significant architectural changes, leading to a `ModuleNotFoundError` when attempting to use `pennylane.qnn.keras`.

### Data Analysis Key Findings
*   The installed Keras version was identified as `3.13.0`.
*   PennyLane's `KerasLayer` (`pennylane.qnn.keras`) is designed for `tf.keras` (Keras 2.x), which is integrated within TensorFlow.
*   Keras 3.x represents a major architectural shift, making it a standalone library incompatible with the `pennylane.qnn.keras` module, resulting in a `ModuleNotFoundError`.

### Insights or Next Steps
*   To resolve the incompatibility, consider downgrading Keras to a 2.x version by installing a compatible TensorFlow version (`tf.keras`).
*   Alternatively, if Keras 3.x is required, explore manual integration of quantum circuits into Keras 3.x models, or await future PennyLane updates that may introduce native Keras 3.x support.
